In [92]:
import pandas as pd
import duckdb
import plotly.express as px
import numpy as np

In [93]:
JSONLINE_OUTPUT = '/home/alextay96/Desktop/workspace/Calvinn-Alex-research-2024/data/gallup/silver/gallup.jsonl'

In [94]:
conn = duckdb.connect()


In [95]:
src_dir = "/home/alextay96/Desktop/workspace/Calvinn-Alex-research-2024/data/gallup/bronze"
df = conn.execute(
    f'''
    Select * 
    from read_parquet('{src_dir}/*.parquet')
    '''
).df()

In [96]:
df.drop_duplicates(subset="url", inplace=True)

In [97]:
df

,url,title,category,published_at,content,root_topic,text_tokens,num_tokens
0,https://news.gallup.com/poll/266807/percentage...,What Percentage of Americans Own Stock?,THE SHORT ANSWER,"MAY 24, 2023",Editor's Note: This article was updated May 24...,Short Answer,"[9597, 596, 7181, 25, 1115, 4652, 574, 6177, 3...",1400
1,https://news.gallup.com/poll/270296/americans-...,Do Americans Like or Dislike 'Big Business'?,THE SHORT ANSWER,"JUNE 27, 2022",Editor's Note: This article was revised on Jul...,Short Answer,"[9597, 596, 7181, 25, 1115, 4652, 574, 32828, ...",419
2,https://news.gallup.com/poll/272765/lowest-gal...,Who Had the Lowest Gallup Presidential Job App...,THE SHORT ANSWER,"DECEMBER 26, 2019","WASHINGTON, D.C. -- Harry Truman registered a ...",Short Answer,"[51577, 11, 423, 732, 13, 1198, 14253, 80936, ...",315
3,https://news.gallup.com/poll/350096/americans-...,Do Americans Believe in UFOs?,THE SHORT ANSWER,"AUGUST 20, 2021",Editor's Note: This article was updated on Aug...,Short Answer,"[9597, 596, 7181, 25, 1115, 4652, 574, 6177, 3...",388
4,https://news.gallup.com/poll/284135/percentage...,What Percentage of Americans Smoke Marijuana?,THE SHORT ANSWER,"AUGUST 15, 2022",Editor's Note: This article was updated on Aug...,Short Answer,"[9597, 596, 7181, 25, 1115, 4652, 574, 6177, 3...",353
...,...,...,...,...,...,...,...,...
3477,https://news.gallup.com/poll/142826/Republican...,Republicans Remain Disproportionately White an...,POLITICS,"SEPTEMBER 1, 2010","PRINCETON, NJ -- About 9 out of 10 Republicans...",Religion,"[6616, 39539, 1372, 715, 11, 27816, 1198, 1018...",983
3478,https://news.gallup.com/poll/145379/Religious-...,Very Religious Americans Lead Healthier Lives,WELLBEING,"DECEMBER 23, 2010",This is the third article in a special multipa...,Religion,"[2028, 374, 279, 4948, 4652, 304, 264, 3361, 6...",1252
3479,https://news.gallup.com/poll/148763/Muslim-Ame...,Most Muslim Americans See No Justification for...,WORLD,"AUGUST 2, 2011",ABU DHABI -- Muslim Americans are the staunche...,Religion,"[1905, 52, 33887, 65025, 1198, 10451, 9053, 52...",1055
3480,https://news.gallup.com/poll/113575/Complex-Re...,The Complex Relationship Between Religion and ...,WORLD,"DECEMBER 24, 2008",Takeaway: Regardless of whether they affiliate...,Religion,"[18293, 14075, 25, 44840, 315, 3508, 814, 2232...",1307


In [98]:
df["num_tokens"].sum() /1e6

2.82917

In [110]:
df.groupby("category")["num_tokens"].sum().sort_values(ascending=False).reset_index().head(20)

,category,num_tokens
0,POLITICS,695223
1,POLLING MATTERS,360524
2,GALLUP PODCAST,297411
3,ECONOMY,285342
4,GALLUP BLOG,279247
5,WORLD,233596
6,SOCIAL & POLICY ISSUES,118185
7,WELLBEING,116440
8,GALLUP VAULT,86748
9,BLOG,78376


In [111]:
df.groupby("category")["num_tokens"].count().sort_values(ascending=False).reset_index().head(20)

,category,num_tokens
0,POLITICS,501
1,GALLUP BLOG,229
2,ECONOMY,218
3,POLLING MATTERS,203
4,GALLUP PODCAST,200
5,WORLD,178
6,GALLUP VAULT,142
7,BLOG,98
8,SOCIAL & POLICY ISSUES,81
9,WELLBEING,72


In [99]:
df.iloc[np.random.randint(0, len(df))]["content"].split("\n")

['STORY HIGHLIGHTS',
 '21% say it is a good time to buy a house, lowest in Gallup’s trend',
 'Prior to 2022, 50% or more consistently thought it was a good time to buy',
 "Significantly fewer expect local housing prices to increase in next yearWASHINGTON, D.C. -- Twenty-one percent of U.S. adults believe it is a good time to buy a house, down nine percentage points from the prior low recorded last year. The 2022 and 2023 readings are the only times that less than half of Americans have perceived the housing market as being good for buyers in Gallup’s trend since 1978.The latest results are from Gallup’s annual Economy and Personal Finance poll, conducted April 3-25. Seventy-eight percent currently say it is a bad time to buy a house.Gallup first asked Americans about their perceptions of the housing market in 1978, when 53% thought it was a good time to buy a house. Thirteen years later, when the question was asked again, 67% held that view. The record high of 81% was recorded in 2003,

In [100]:

jsonl_string = df.to_json(orient='records', lines=True)

with open(JSONLINE_OUTPUT, 'w') as file:
    file.write(jsonl_string)

In [103]:
df_2 = conn.execute(
    f'''
    Select * 
    from read_json_auto('/home/alextay96/Desktop/workspace/Calvinn-Alex-research-2024/data/gallup/silver/gallup.jsonl')
    '''
).df()
df_2

,url,title,category,published_at,content,root_topic,text_tokens,num_tokens
0,https://news.gallup.com/poll/266807/percentage...,What Percentage of Americans Own Stock?,THE SHORT ANSWER,"MAY 24, 2023",Editor's Note: This article was updated May 24...,Short Answer,"[9597, 596, 7181, 25, 1115, 4652, 574, 6177, 3...",1400
1,https://news.gallup.com/poll/270296/americans-...,Do Americans Like or Dislike 'Big Business'?,THE SHORT ANSWER,"JUNE 27, 2022",Editor's Note: This article was revised on Jul...,Short Answer,"[9597, 596, 7181, 25, 1115, 4652, 574, 32828, ...",419
2,https://news.gallup.com/poll/272765/lowest-gal...,Who Had the Lowest Gallup Presidential Job App...,THE SHORT ANSWER,"DECEMBER 26, 2019","WASHINGTON, D.C. -- Harry Truman registered a ...",Short Answer,"[51577, 11, 423, 732, 13, 1198, 14253, 80936, ...",315
3,https://news.gallup.com/poll/350096/americans-...,Do Americans Believe in UFOs?,THE SHORT ANSWER,"AUGUST 20, 2021",Editor's Note: This article was updated on Aug...,Short Answer,"[9597, 596, 7181, 25, 1115, 4652, 574, 6177, 3...",388
4,https://news.gallup.com/poll/284135/percentage...,What Percentage of Americans Smoke Marijuana?,THE SHORT ANSWER,"AUGUST 15, 2022",Editor's Note: This article was updated on Aug...,Short Answer,"[9597, 596, 7181, 25, 1115, 4652, 574, 6177, 3...",353
...,...,...,...,...,...,...,...,...
2163,https://news.gallup.com/poll/142826/Republican...,Republicans Remain Disproportionately White an...,POLITICS,"SEPTEMBER 1, 2010","PRINCETON, NJ -- About 9 out of 10 Republicans...",Religion,"[6616, 39539, 1372, 715, 11, 27816, 1198, 1018...",983
2164,https://news.gallup.com/poll/145379/Religious-...,Very Religious Americans Lead Healthier Lives,WELLBEING,"DECEMBER 23, 2010",This is the third article in a special multipa...,Religion,"[2028, 374, 279, 4948, 4652, 304, 264, 3361, 6...",1252
2165,https://news.gallup.com/poll/148763/Muslim-Ame...,Most Muslim Americans See No Justification for...,WORLD,"AUGUST 2, 2011",ABU DHABI -- Muslim Americans are the staunche...,Religion,"[1905, 52, 33887, 65025, 1198, 10451, 9053, 52...",1055
2166,https://news.gallup.com/poll/113575/Complex-Re...,The Complex Relationship Between Religion and ...,WORLD,"DECEMBER 24, 2008",Takeaway: Regardless of whether they affiliate...,Religion,"[18293, 14075, 25, 44840, 315, 3508, 814, 2232...",1307
